# Importing needed libraries 


In [5]:
from flair.models import TextClassifier
from flair.data import Sentence
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import spacy
nlp_md = spacy.load('en_core_web_md')
import plotly.express as px
import pickle
from nltk.tokenize import RegexpTokenizer

# Defining Functions 

In [1]:

# calcualte Major and Minor sentiment and vizualize it in a dashabord style

def Display_sentiment(df_sentiment  , booktitle = ' book title'):
  warnings.filterwarnings("always")

  l = df_sentiment.iloc[2,:]
  l = pd.DataFrame(l) 
  l['Major'] = l.iloc[:,0].rolling(40).mean()
  l['Minor'] = l.iloc[:,0].rolling(10).mean()

  fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(30, 15))
  axes[0].set_title((str(booktitle) + ' : Sentimental Tendency') , fontsize = 30  , loc='left')
  axes[0].plot(l['Major'])
  axes[0].set_xlabel('Sentences' , fontsize=20)
  axes[0].set_ylabel('Sentiment', fontsize=20)
  axes[0].legend(['Major Sentiment(rolling average for 30 sentences)'], fontsize=20)
  axes[0].fill_between(l.index, l['Major'] ,0 , where=l['Major'] >= 0, facecolor='lightgreen',  interpolate=True)
  axes[0].fill_between(l.index, l['Major'] ,0 , where=l['Major'] <= 0, facecolor='lightcoral',  interpolate=True)
  axes[0].set_facecolor('white')
  axes[1].plot(l['Minor'])
  axes[1].set_xlabel('Sentences' , fontsize=15)
  axes[1].set_ylabel('Sentiment' , fontsize=15)
  axes[1].fill_between(l.index, l['Minor'] ,0 , where=l['Minor'] >= 0, facecolor='lightgreen',  interpolate=True)
  axes[1].fill_between(l.index, l['Minor'] ,0 , where=l['Minor'] <= 0, facecolor='lightcoral',  interpolate=True)
  axes[1].set_facecolor('white')
  axes[1].legend(['Minor Sentiment(rolling average for 5 sentences)'] , fontsize=20)


# perform calculations on df_emotions and display results 
def Display_emotions(df_emotions , booktitle = 'Book Title'):
  
  l = df_emotions.T.value_counts().index
  l = l.tolist()
  v = df_emotions.T.value_counts()
  v = v.tolist()
  fig = go.Figure(data=go.Scatterpolar(r=v,theta=l,fill='toself'))
  fig.update_layout(polar=dict(radialaxis=dict(visible=False),),showlegend=False ,   title={
        'text': (str(booktitle)+' : Composition of Emotion ')})
  fig.show()


  df_emotions1 = df_emotions.T
  df_emotions1['Sentences(50)'] =  ((df_emotions1.index/50).astype(int))*50
  df_emotions1['1'] = df_emotions1.index
  data = df_emotions1.groupby(by=['Sentences(50)',0]).count()
  data = pd.DataFrame(data)
  data.reset_index(inplace=True)  
  data.columns = ['Sentences','Emotion','Count']
  fig = px.line(data,
                 x=	'Sentences' ,  y='Count', color='Emotion',
                 title=((str(booktitle)+" : Emotional Journey")))
  fig.show()


  
#the Below Function draw a Sunburt chart displaying entities mentioned alog with their types 

def Display_entities(df_ents , booktitle = 'Booktitle'):

  # replace label with more understandble labels :
  df_ents = df_ents.replace('NORP', 'Nationalities/groups')
  df_ents = df_ents.replace('PERSON', 'People')
  df_ents = df_ents.replace('GPE', 'Countries/Cities')
  df_ents = df_ents.replace('LOC', 'Locations')
  df_ents = df_ents.replace('EVENT', 'Event')
  df_ents = df_ents.replace('LAW', 'Laws')

  # FIRST FIGURE
  df_ents1 = df_ents
  df_ents1 = df_ents1.groupby(by=['label','entity']).count().sort_values(by='index', ascending=False).head(20)
  df_ents1 = df_ents1.reset_index(level=['label', 'entity'])
  df = px.data.tips()
  fig = px.sunburst(df_ents1, path=['label', 'entity'], values='index' , title=( str(booktitle)+" : Mentions of Entities (locations, nationalities, events, and characters)"))
  fig.update_layout(width=1000,height=900)
  fig.show()

  #SECOND FIGURE
  df_ents2 = df_ents
  df_ents2['sentence'] = ((df_ents2.index/50).astype(int))*50
  df_ents2 = df_ents2.groupby(by=['sentence','label','entity']).count()
  df_ents2 = df_ents2.reset_index(level=['sentence','label','entity'])
  df_ents2 = df_ents2[df_ents2['index'] > 1]
  df_ents2.columns = ['sentence(every 50 senteces)', 'label', 'entity', 'count']
  fig = px.scatter(df_ents2,x='sentence(every 50 senteces)', y="entity",
	         size="count", color="entity")
  fig.show()
